In [ ]:
import argparse
import cv2
import numpy as np
import os

In [ ]:
parser = argparse.ArgumentParser(description="Camera Intrinsic Calibration")
parser.add_argument('-input', '--INPUT_TYPE', default='camera', type=str, help='Input Source: camera/video/image')
parser.add_argument('-type', '--CAMERA_TYPE', default='fisheye', type=str, help='Camera Type: fisheye/normal')
parser.add_argument('-id', '--CAMERA_ID', default=1, type=int, help='Camera ID')
parser.add_argument('-path', '--INPUT_PATH', default='./data/', type=str, help='Input Video/Image Path')
parser.add_argument('-video', '--VIDEO_FILE', default='video.mp4', type=str, help='Input Video File Name (eg.:video.mp4)')
parser.add_argument('-image', '--IMAGE_FILE', default='img_raw', type=str, help='Input Image File Name Prefix (eg.:img_raw)')
parser.add_argument('-mode', '--SELECT_MODE', default='auto', type=str, help='Image Select Mode: auto/manual')
parser.add_argument('-fw','--FRAME_WIDTH', default=1280, type=int, help='Camera Frame Width')
parser.add_argument('-fh','--FRAME_HEIGHT', default=720, type=int, help='Camera Frame Height')
parser.add_argument('-bw','--BORAD_WIDTH', default=7, type=int, help='Chess Board Width (corners number)')
parser.add_argument('-bh','--BORAD_HEIGHT', default=6, type=int, help='Chess Board Height (corners number)')
parser.add_argument('-size','--SQUARE_SIZE', default=10, type=int, help='Chess Board Square Size (mm)')
parser.add_argument('-num','--CALIBRATE_NUMBER', default=5, type=int, help='Least Required Calibration Frame Number')
parser.add_argument('-delay','--FRAME_DELAY', default=8, type=int, help='Capture Image Time Interval (frame number)')
parser.add_argument('-subpix','--SUBPIX_REGION', default=5, type=int, help='Corners Subpix Optimization Region')
parser.add_argument('-fs', '--FOCAL_SCALE', default=1, type=float, help='Camera Undistort Focal Scale')
parser.add_argument('-ss', '--SIZE_SCALE', default=2, type=float, help='Camera Undistort Size Scale')
parser.add_argument('-store','--STORE_CAPTURE', default=False, type=bool, help='Store Captured Images (Ture/False)')
parser.add_argument('-crop','--FRAME_CROP', default=False, type=bool, help='Crop Input Video/Image to (fw,fh) (Ture/False)')
parser.add_argument('-resize','--FRAME_RESIZE', default=False, type=bool, help='Resize Input Video/Image to (fw,fh) (Ture/False)')
cfgs = parser.parse_args([])                 # Jupyter Notebook中直接运行时要加[], py文件则去掉

In [ ]:
# cfgs.INPUT_TYPE = 'image'                  # 输入形式 相机/视频/图像
# cfgs.CAMERA_TYPE = 'normal'                # 相机类型 鱼眼/普通
# cfgs.CAMERA_ID = 0                         # 相机编号
# cfgs.INPUT_PATH = './data/'                # 输入路径
# cfgs.VIDEO_FILE = 'video.mp4'              # 输入视频文件名(含扩展名)
# cfgs.IMAGE_FILE = 'raw'                    # 输入图像文件名前缀
# cfgs.SELECT_MODE = 'manual'                # 选择自动/手动模式
# cfgs.FRAME_WIDTH = 1280                    # 相机分辨率 帧宽度
# cfgs.FRAME_HEIGHT = 720                    # 相机分辨率 帧高度
# cfgs.BORAD_WIDTH = 9                       # 棋盘宽度 【内角点数】
# cfgs.BORAD_HEIGHT = 6                      # 棋盘高度 【内角点数】
# cfgs.SQUARE_SIZE = 10                      # 棋盘格边长 mm
# cfgs.CALIBRATE_NUMBER = 5                 # 最小标定图片采样数量
# cfgs.FRAME_DELAY = 15                      # 延时的帧数
# cfgs.STORE_CAPTURE = True                  # 是否保存抓取的图像
# cfgs.FRAME_CROP = True                     # 是否将输入视频/图像尺寸裁剪至FRAME_WIDTH和FRAME_HEIGHT的设定值
# cfgs.FRAME_RESIZE = True                   # 是否将输入视频/图像尺寸缩放至FRAME_WIDTH和FRAME_HEIGHT的设定值（图像缩放会改变相机焦距）

In [ ]:
CHESS_BOARD_PATTERN = (cfgs.BORAD_WIDTH, cfgs.BORAD_HEIGHT)                 # 棋盘格图案布局
BOARD = np.array([ [(j * cfgs.SQUARE_SIZE, i * cfgs.SQUARE_SIZE, 0.)]
                  for i in range(cfgs.BORAD_HEIGHT) for j in range(cfgs.BORAD_WIDTH) ],dtype=np.float32)     # 棋盘角点二维坐标（乘上尺寸）

In [ ]:
class CalibData:                             # 标定数据类
    def __init__(self):
        self.type = None                     # 自定义数据类型
        self.camera_mat = None               # 相机内参
        self.dist_coeff = None               # 畸变参数
        self.rvecs = None                    # 旋转向量
        self.tvecs = None                    # 平移向量
        self.map1 = None                     # 映射矩阵1
        self.map2 = None                     # 映射矩阵2
        self.reproj_err = None               # 重投影误差
        self.ok = False                      # 数据采集完成标志

In [ ]:
# cv2.findChessboardCorners ( image,         # 棋盘图像
#                             patternSize,   # 棋盘格行和列的【内角点】数量
#                             corners,       # 输出数组
#                             flags          # 操作标志
#                             )
# flags:
#     CV_CALIB_CB_ADAPTIVE_THRESH            # 使用自适应阈值处理将图像转换为黑白图像
#     CV_CALIB_CB_NORMALIZE_IMAGE            # 对图像进行归一化。
#     CV_CALIB_CB_FILTER_QUADS               # 过滤在轮廓检索阶段提取的假四边形。
#     CALIB_CB_FAST_CHECK                    # 对查找棋盘角的图像进行快速检查

In [ ]:
# cv2.cornerSubPix (image,                   # 棋盘图像
#                   corners,                 # 棋盘角点
#                   winSize,                 # 搜索窗口边长的一半
#                   zeroZone,                # 搜索区域死区大小的一半, (-1,-1)代表无
#                   criteria                 # 迭代停止标准
#                  )

In [ ]:
class CornerData:                            # 棋盘寻找和亚像素优化 得到角点坐标
    def __init__(self, raw_frame):
        self.raw_frame = raw_frame
        self.corners = None
        self.ok = False
        # 寻找棋盘角点
        self.ok, self.corners = cv2.findChessboardCorners(self.raw_frame, CHESS_BOARD_PATTERN,
                                flags = cv2.CALIB_CB_ADAPTIVE_THRESH|cv2.CALIB_CB_NORMALIZE_IMAGE|cv2.CALIB_CB_FAST_CHECK)
        if not self.ok: return
        # 角点坐标亚像素优化
        gray = cv2.cvtColor(self.raw_frame, cv2.COLOR_BGR2GRAY)
        self.corners = cv2.cornerSubPix(gray, self.corners, (11, 11), (-1, -1),
                        (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01))
        # 在图上显示角点
        cv2.drawChessboardCorners(self.raw_frame, CHESS_BOARD_PATTERN, self.corners, self.ok)

In [ ]:
# cv2.fisheye.calibrate ( objectPoints,      # 角点在棋盘中的空间坐标向量
#                         imagePoints,       # 角点在图像中的坐标向量
#                         image_size,        # 图片大小
#                         K,                 # 相机内参矩阵
#                         D,                 # 畸变参数向量
#                         rvecs,             # 旋转向量
#                         tvecs,             # 平移向量
#                         flags,             # 操作标志
#                         criteria           # 迭代优化算法的停止标准
#                         )

# flags:
#     cv2.fisheye.CALIB_USE_INTRINSIC_GUESS     # 当相机内参矩阵包含有效的fx，fy，cx，cy初始值时，这些值会进一步进行优化
#                                               # 否则，（cx，cy）初始化设置为图像中心（使用imageSize），并且以最小二乘法计算焦距。
#     cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC     # 在每次内部参数优化迭代之后，将重新计算外部参数。
#     cv2.fisheye.CALIB_CHECK_COND              # 检查条件编号的有效性
#     cv2.fisheye.CALIB_FIX_SKEW                # 偏斜系数（alpha）设置为零，并保持为零
#     cv2.fisheye.CALIB_FIX_K1 (K1-K4)          # 选定的畸变系数设置为零，并保持为零  CALIB_FIX_INTRINSIC则全为零

# criteria:
#     TermCriteria (int type, int maxCount, double epsilon)      # 类型、最大计数次数、最小精度
#     Criteria type, can be one of: COUNT, EPS or COUNT + EPS    # 角点优化最大迭代次数或角点优化位置移动量小于epsilon值

In [ ]:
# cv2.checkRange 检查元素非空及无异常值

In [ ]:
class Fisheye:
    def __init__(self):
        self.data = CalibData()
        self.inited = False
    
    # 更新标定数据，分为初始化和精调
    def update(self, corners, frame_size):
        board = [BOARD] * len(corners)
        if not self.inited:
            self._update_init(board, corners, frame_size)
            self.inited = True
        else:
            self._update_refine(board, corners, frame_size)
        self._calc_reproj_err(corners)
        
    # 得到一定数量标定样本时进行初始标定
    def _update_init(self, board, corners, frame_size):
        data = self.data
        data.type = "FISHEYE"
        data.camera_mat = np.eye(3, 3)
        data.dist_coeff = np.zeros((4, 1))
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.fisheye.calibrate(
            board, corners, frame_size, data.camera_mat, data.dist_coeff,
            flags=cv2.fisheye.CALIB_FIX_SKEW|cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC,
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 1e-6)) 
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)

    # 精调时启用CALIB_USE_INTRINSIC_GUESS
    def _update_refine(self, board, corners, frame_size):
        data = self.data
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.fisheye.calibrate(
            board, corners, frame_size, data.camera_mat, data.dist_coeff,
            flags=cv2.fisheye.CALIB_FIX_SKEW|cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC|cv2.CALIB_USE_INTRINSIC_GUESS,
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 10, 1e-6))
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)

    # 计算重投影误差，单位为像素
    def _calc_reproj_err(self, corners):
        if not self.inited: return
        data = self.data
        data.reproj_err = []
        for i in range(len(corners)):
            corners_reproj, _ = cv2.fisheye.projectPoints(BOARD, data.rvecs[i], data.tvecs[i], data.camera_mat, data.dist_coeff)
            err = cv2.norm(corners_reproj, corners[i], cv2.NORM_L2) / len(corners_reproj)
            data.reproj_err.append(err)

In [ ]:
# cv2.calibrateCamera (objectPoints,         # 角点在棋盘中的空间坐标向量        
#                      imagePoints,          # 角点在图像中的坐标向量
#                      image_size,           # 图片大小
#                      K,                    # 相机内参矩阵
#                      D,                    # 畸变参数向量
#                      rvecs,                # 旋转向量
#                      tvecs,                # 平移向量
#                      flags,                # 操作标志
#                      criteria              # 迭代优化算法的停止标准
#                     )

# flags:
#     cv2.CALIB_USE_INTRINSIC_GUESS          # 当相机内参矩阵包含有效的fx，fy，cx，cy初始值时，这些值会进一步进行优化
#                                            # 否则，（cx，cy）初始化设置为图像中心（使用imageSize），并且以最小二乘法计算焦距
#     cv2.CALIB_FIX_PRINCIPAL_POINT          # 固定光轴点（当设置CALIB_USE_INTRINSIC_GUESS时可以使用）
#     cv2.CALIB_FIX_ASPECT_RATIO             # 固定fx/fy的值，函数仅将fy视为自由参数
#     cv2.CALIB_ZERO_TANGENT_DIST            # 切向畸变系数（p1，p2） 设置为零并保持为零
#     cv2.CALIB_FIX_FOCAL_LENGTH             # 如果设置了CALIB_USE_INTRINSIC_GUESS，则在全局优化过程中不会更改焦距
#     cv2.CALIB_FIX_K1 (K1-K6)               # 固定相应的径向畸变系数为0或给定的初始值
#     cv2.CALIB_RATIONAL_MODEL               # 理想模型：启用系数k4，k5和k6。此时返回8个或更多的系数
#     cv2.CALIB_THIN_PRISM_MODEL             # 薄棱镜模型：启用系数s1，s2，s3和s4。此时返回12个或更多的系数
#     cv2.CALIB_FIX_S1_S2_S3_S4              # 固定薄棱镜畸变系数为0或给定的初始值
#     cv2.CALIB_TILTED_MODEL                 # 倾斜模型：启用系数tauX和tauY。此时返回14个系数
#     cv2.CALIB_FIX_TAUX_TAUY                # 固定倾斜传感器模型的系数为0或给定的初始值

In [ ]:
class Normal:
    def __init__(self):
        self.data = CalibData()
        self.inited = False
        
    def update(self, corners, frame_size):
        board = [BOARD] * len(corners)
        if not self.inited:
            self._update_init(board, corners, frame_size)
            self.inited = True
        else:
            self._update_refine(board, corners, frame_size)
        self._calc_reproj_err(corners)
        
    def _update_init(self, board, corners, frame_size):
        data = self.data
        data.type = "NORMAL"
        data.camera_mat = np.eye(3, 3)
        data.dist_coeff = np.zeros((5, 1))     # 畸变向量大小，根据使用模型修改
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.calibrateCamera(
            board, corners, frame_size, data.camera_mat, data.dist_coeff, 
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 1e-6))
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)
        
    def _update_refine(self, board, corners, frame_size):
        data = self.data
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.calibrateCamera(
            board, corners, frame_size, data.camera_mat, data.dist_coeff,  
            flags = cv2.CALIB_USE_INTRINSIC_GUESS,
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 10, 1e-6))
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)
        
    def _calc_reproj_err(self, corners):
        if not self.inited: return
        data = self.data
        data.reproj_err = []
        for i in range(len(corners)):
            corners_reproj, _ = cv2.projectPoints(BOARD, data.rvecs[i], data.tvecs[i], data.camera_mat, data.dist_coeff)
            err = cv2.norm(corners_reproj, corners[i], cv2.NORM_L2) / len(corners_reproj)
            data.reproj_err.append(err)

In [ ]:
class History:                               # 定义历史存储数据类及相应操作
    def __init__(self):
        self.corners = []
        self.updated = False
    def append(self, current):
        if not current.ok: return
        self.corners.append(current.corners)
        self.updated = True
    def removei(self, i):
        if not 0 <= i < len(self): return
        del self.corners[i]
        self.updated = True
    def __len__(self):
        return len(self.corners)
    def get_corners(self):
        self.updated = False
        return self.corners

In [ ]:
class Flags:                                      # 标定数据类
    def __init__(self):
        self.READ_FAIL_CTR = 0                    # 读取超时计数
        self.frame_id = 0                         # 读入帧数的ID
        self.ready = False                        # 是否可以开始标定

In [ ]:
# cv2.fisheye.initUndistortRectifyMap (K,         # 相机内参矩阵
#                                      D,         # 畸变向量
#                                      R,         # 旋转矩阵
#                                      P,         # 新的相机矩阵
#                                      size,      # 输出图像大小
#                                      m1type,    # 映射矩阵类型
#                                      map1,      # 输出映射矩阵1
#                                      map2       # 输出映射矩阵2
#                                     )

In [ ]:
def main():
    flags = Flags()                                                              # 标志位初始化
    history = History()                                                          # 存储数据类
    
    if cfgs.CAMERA_TYPE == 'fisheye':
        camera = Fisheye()                                                       # 鱼眼相机类
    elif cfgs.CAMERA_TYPE == 'normal':
        camera = Normal()                                                        # 普通相机类
    else:
        raise Exception("CAMERA TYPE should be fisheye or normal")
    
    if cfgs.INPUT_TYPE == 'camera':                                              # 相机输入模式
        cap = cv2.VideoCapture(cfgs.CAMERA_ID)                                   # 开启相机
        if not cap.isOpened(): 
            raise Exception("camera {} open failed".format(cfgs.CAMERA_ID))      # 开启失败报错
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, cfgs.FRAME_WIDTH)                      # 设置相机分辨率
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cfgs.FRAME_HEIGHT)
    elif cfgs.INPUT_TYPE == 'video':                                             # 视频输入模式
        cap = cv2.VideoCapture(cfgs.INPUT_PATH + cfgs.VIDEO_FILE)                # 读取视频
        if not cap.isOpened(): 
            raise Exception("from {} read video failed".format(cfgs.INPUT_PATH + cfgs.VIDEO_FILE))
    elif cfgs.INPUT_TYPE == 'image':                                             # 图片输入模式
        filePath = [os.path.join(cfgs.INPUT_PATH, x) for x in os.listdir(cfgs.INPUT_PATH) 
                    if any(x.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])
                   ]                                                             # 得到给定路径下所有图片文件
        filenames = [filename for filename in filePath if cfgs.IMAGE_FILE in filename]    # 再筛选出包含给定名字的图片
        if len(filenames) == 0:
            raise Exception("from {} read images failed".format(cfgs.INPUT_PATH))
    else:
        raise Exception("INPUT TYPE should be camera, video or image")
        
    if cfgs.INPUT_TYPE == 'image':                                                # 图片输入标定
        for filename in filenames:
            print(filename)
            raw_frame = cv2.imread(filename)
            if cfgs.FRAME_CROP:                                                   # 裁剪图片尺寸
                if raw_frame.shape[1]-cfgs.FRAME_WIDTH < 0 or raw_frame.shape[0]-cfgs.FRAME_HEIGHT < 0:
                    raise Exception("CROP size should be smaller than original size")
                raw_frame = raw_frame[
                    round((raw_frame.shape[0]-cfgs.FRAME_HEIGHT)/2):round((raw_frame.shape[0]-cfgs.FRAME_HEIGHT)/2)+cfgs.FRAME_HEIGHT,
                    round((raw_frame.shape[1]-cfgs.FRAME_WIDTH)/2):round((raw_frame.shape[1]-cfgs.FRAME_WIDTH)/2)+cfgs.FRAME_WIDTH ]           
            elif cfgs.FRAME_RESIZE:                                               # 缩放图片尺寸
                raw_frame = cv2.resize(raw_frame, (cfgs.FRAME_WIDTH, cfgs.FRAME_HEIGHT))
            current = CornerData(raw_frame)                                       # 得到棋盘角点坐标
            display = "raw_frame: press SPACE to SELECT, other key to SKIP, press ESC to QUIT"
            cv2.namedWindow(display, flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
            cv2.imshow(display, raw_frame)
            if len(history) > cfgs.CALIBRATE_NUMBER:                              # 重映射输出 去畸变图像
                undist_frame = cv2.remap(raw_frame, calib.map1, calib.map2, cv2.INTER_LINEAR)
                cv2.namedWindow("undist_frame", flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
                cv2.imshow("undist_frame", undist_frame)                          
            key = cv2.waitKey(1)
            if cfgs.SELECT_MODE == 'manual':                                      # 手动操作模式 空格挑选图片
                key = cv2.waitKey(0)
            if key == 32 or cfgs.SELECT_MODE == 'auto':
                history.append(current)                                           # 存储数据
                if len(history) >= cfgs.CALIBRATE_NUMBER:                         # 达到标定采样数量时开始标定
                    camera.update(history.get_corners(), raw_frame.shape[1::-1])  # 更新标定数据
                    calib = camera.data                                           # 取出标定参数 K D等     
                    if cfgs.CAMERA_TYPE == 'fisheye':                             # 得到去畸变映射map矩阵 鱼眼和普通相机函数不同
                        calib.map1, calib.map2 = cv2.fisheye.initUndistortRectifyMap(
                            calib.camera_mat, calib.dist_coeff, np.eye(3, 3), calib.camera_mat, raw_frame.shape[1::-1], cv2.CV_16SC2)
                    else:
                        calib.map1, calib.map2 = cv2.initUndistortRectifyMap(
                            calib.camera_mat, calib.dist_coeff, np.eye(3, 3), calib.camera_mat, raw_frame.shape[1::-1], cv2.CV_16SC2)
            if key == 27: break                                                   # esc退出
    else:
        while True:                                                               # 视频输入标定
            key = cv2.waitKey(1)                                                  # 获取键盘输入
            ok, raw_frame = cap.read()                                            # 从相机读入原始帧
            if not ok:
                if cfgs.INPUT_TYPE == 'video': break
                flags.READ_FAIL_CTR += 1
                if flags.READ_FAIL_CTR >= 20:                                     # 读取视频超时
                    raise Exception("video read failed")
            else:
                flags.READ_FAIL_CTR = 0
                flags.frame_id += 1
                
            if cfgs.FRAME_CROP:                                                   # 裁剪视频尺寸
                if raw_frame.shape[1]-cfgs.FRAME_WIDTH < 0 or raw_frame.shape[0]-cfgs.FRAME_HEIGHT < 0:
                    raise Exception("CROP size should be smaller than original size")
                raw_frame = raw_frame[
                    round((raw_frame.shape[0]-cfgs.FRAME_HEIGHT)/2):round((raw_frame.shape[0]-cfgs.FRAME_HEIGHT)/2)+cfgs.FRAME_HEIGHT,
                    round((raw_frame.shape[1]-cfgs.FRAME_WIDTH)/2):round((raw_frame.shape[1]-cfgs.FRAME_WIDTH)/2)+cfgs.FRAME_WIDTH ]           
            elif cfgs.FRAME_RESIZE:                                               # 缩放视频尺寸
                raw_frame = cv2.resize(raw_frame, (cfgs.FRAME_WIDTH, cfgs.FRAME_HEIGHT))
            
            if key == 32 or (cfgs.INPUT_TYPE == 'video' and cfgs.SELECT_MODE == 'auto'):
                flags.ready = True                                                # 按空格开始标定 视频自动模式则直接置为True
            
            if cfgs.SELECT_MODE == 'auto' and flags.ready and flags.frame_id % cfgs.FRAME_DELAY == 0:   # 间隔一定帧数采样
                if cfgs.STORE_CAPTURE:
                    cv2.imwrite('./data/img_raw{}.jpg'.format(len(history)),raw_frame)      # 存储该帧图像
                current = CornerData(raw_frame)                                   # 得到棋盘角点坐标
                history.append(current)                                           # 存储数据
                print(len(history))                                               # 显示目前采集数据数量

            if cfgs.SELECT_MODE == 'manual' and key == 32:                        # 手动模式下按空格键采集该帧图像进行标定                                            
                if cfgs.STORE_CAPTURE:
                    cv2.imwrite('./data/img_raw{}.jpg'.format(len(history)),raw_frame)      # 存储该帧图像
                current = CornerData(raw_frame)                                   # 得到棋盘角点坐标
                history.append(current)                                           # 存储数据
                print(len(history))                                               # 显示目前采集数据数量
                
            if flags.ready and len(history) >= cfgs.CALIBRATE_NUMBER and history.updated:   # 达到标定采样数量且数据更新时开始标定
                camera.update(history.get_corners(), raw_frame.shape[1::-1])      # 更新标定参数
                calib = camera.data                                               # 取出标定参数 K D等                    
                if cfgs.CAMERA_TYPE == 'fisheye':                                 # 得到去畸变映射map矩阵 鱼眼和普通相机函数不同
                    calib.map1, calib.map2 = cv2.fisheye.initUndistortRectifyMap(
                        calib.camera_mat, calib.dist_coeff, np.eye(3, 3), calib.camera_mat, raw_frame.shape[1::-1], cv2.CV_16SC2)
                else:
                    calib.map1, calib.map2 = cv2.initUndistortRectifyMap(
                        calib.camera_mat, calib.dist_coeff, np.eye(3, 3), calib.camera_mat, raw_frame.shape[1::-1], cv2.CV_16SC2)

            if flags.ready and len(history) >= cfgs.CALIBRATE_NUMBER:
                undist_frame = cv2.remap(raw_frame, calib.map1, calib.map2, cv2.INTER_LINEAR); 
                cv2.namedWindow("undist_frame", flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
                cv2.imshow("undist_frame", undist_frame)                          # 重映射输出 去畸变图像
            
            if cfgs.SELECT_MODE == 'manual':
                display = "raw_frame: press SPACE to capture image"
            elif cfgs.INPUT_TYPE == 'camera':
                display = "raw_frame: press SPACE to start calibration"
            else:
                display = "raw_frame"
            cv2.namedWindow(display, flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
            cv2.imshow(display, raw_frame)
            if key == 27: break                                                   # ESC退出
            
        cap.release()
        
    cv2.destroyAllWindows() 
    
    if len(history) == 0:
        raise Exception("Calibration failed. Chessboard not found, check the parameters")  # 标定失败 未找到棋盘或参数设置错误
    if len(history) < cfgs.CALIBRATE_NUMBER:
        raise Exception("Warning: Calibration images are not enough. At least {} images are needed.".format(cfgs.CALIBRATE_NUMBER))            

    print("Calibration Complete")
    print("Camera Matrix is : {}".format(camera.data.camera_mat.tolist()))                 # 相机内参
    print("Distortion Coefficient is : {}".format(camera.data.dist_coeff.tolist()))        # 畸变向量
    print("Reprojection Error is : {}".format(np.mean(camera.data.reproj_err)))            # 平均重投影误差
    np.save('camera_{}_K.npy'.format(cfgs.CAMERA_ID),camera.data.camera_mat.tolist())
    np.save('camera_{}_D.npy'.format(cfgs.CAMERA_ID),camera.data.dist_coeff.tolist())      # 输出并存储数据
    
if __name__ == '__main__':
    main()
